In [1]:
import csv
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

In [2]:

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL ICP10") \
    .getOrCreate()

In [3]:
#question 1  part1
df = spark.read.csv(r"survey.csv",header=True)
df.createOrReplaceTempView("Survey")

In [4]:
#question 1  part2
#df.write.option("header", "true").csv("spark_survey12.csv")

In [5]:
df

DataFrame[Timestamp: string, Age: string, Gender: string, Country: string, state: string, self_employed: string, family_history: string, treatment: string, work_interfere: string, no_employees: string, remote_work: string, tech_company: string, benefits: string, care_options: string, wellness_program: string, seek_help: string, anonymity: string, leave: string, mental_health_consequence: string, phys_health_consequence: string, coworkers: string, supervisor: string, mental_health_interview: string, phys_health_interview: string, mental_vs_physical: string, obs_consequence: string, comments: string]

In [6]:
df.count()

1259

In [7]:
#question 1  part3
df=df.dropDuplicates()

In [8]:
df.count()

1259

In [9]:
df.groupBy(df.columns).count().where(f.col('count') > 1).select(f.sum('count')).show()

+----------+
|sum(count)|
+----------+
|      null|
+----------+



In [10]:
spark.sql("select * from Survey where Gender = 'Male' or Gender = 'M' or Gender='male'").createTempView("Table_Male")

In [11]:
spark.sql("select * from Survey where Gender = 'Female' or Gender = 'female'").createTempView("Table_Female")

In [12]:
#question 1  part4
spark.sql("select * from Table_Male union select * from Table_Female order by Country").show(100)

+-------------------+---+------+--------------------+-----+-------------+--------------+---------+--------------+--------------+-----------+------------+----------+------------+----------------+----------+----------+------------------+-------------------------+-----------------------+------------+------------+-----------------------+---------------------+------------------+---------------+--------------------+
|          Timestamp|Age|Gender|             Country|state|self_employed|family_history|treatment|work_interfere|  no_employees|remote_work|tech_company|  benefits|care_options|wellness_program| seek_help| anonymity|             leave|mental_health_consequence|phys_health_consequence|   coworkers|  supervisor|mental_health_interview|phys_health_interview|mental_vs_physical|obs_consequence|            comments|
+-------------------+---+------+--------------------+-----+-------------+--------------+---------+--------------+--------------+-----------+------------+----------+--------

In [13]:
#question 1  part5
spark.sql("select treatment,count(*) as count from Survey group by treatment").show()

+---------+-----+
|treatment|count|
+---------+-----+
|       No|  622|
|      Yes|  637|
+---------+-----+



In [14]:
# Question2

In [15]:
# Question2 part1  Join
spark.sql("select m.age,m.Country,m.Gender, m.treatment,f.Gender,f.treatment from Table_Male m join Table_Female f on m.Country = f.Country").show()

+---+-------------+------+---------+------+---------+
|age|      Country|Gender|treatment|Gender|treatment|
+---+-------------+------+---------+------+---------+
| 44|United States|     M|       No|Female|      Yes|
| 44|United States|     M|       No|Female|      Yes|
| 44|United States|     M|       No|female|       No|
| 44|United States|     M|       No|female|      Yes|
| 44|United States|     M|       No|Female|      Yes|
| 44|United States|     M|       No|Female|      Yes|
| 44|United States|     M|       No|Female|      Yes|
| 44|United States|     M|       No|Female|      Yes|
| 44|United States|     M|       No|female|      Yes|
| 44|United States|     M|       No|female|      Yes|
| 44|United States|     M|       No|female|      Yes|
| 44|United States|     M|       No|Female|       No|
| 44|United States|     M|       No|Female|      Yes|
| 44|United States|     M|       No|Female|      Yes|
| 44|United States|     M|       No|female|      Yes|
| 44|United States|     M|  

In [16]:
# Question2 part1  Aggregate Function
spark.sql("select sum(Age),count(Gender) from Survey").show()

+------------------------+-------------+
|sum(CAST(Age AS DOUBLE))|count(Gender)|
+------------------------+-------------+
|        1.00000038724E11|         1259|
+------------------------+-------------+



In [17]:
# Question2 part1  Join
spark.sql("select m.age,m.Country,m.Gender, m.treatment,f.Gender,f.treatment from Table_Male m left join Table_Female f on m.State = f.State").show()

+---+-------------+------+---------+------+---------+
|age|      Country|Gender|treatment|Gender|treatment|
+---+-------------+------+---------+------+---------+
| 44|United States|     M|       No|female|      Yes|
| 44|United States|     M|       No|Female|      Yes|
| 32|       Canada|  Male|       No|female|       No|
| 32|       Canada|  Male|       No|Female|      Yes|
| 32|       Canada|  Male|       No|Female|       No|
| 32|       Canada|  Male|       No|female|      Yes|
| 32|       Canada|  Male|       No|female|      Yes|
| 32|       Canada|  Male|       No|Female|      Yes|
| 32|       Canada|  Male|       No|Female|       No|
| 32|       Canada|  Male|       No|female|      Yes|
| 32|       Canada|  Male|       No|Female|       No|
| 32|       Canada|  Male|       No|Female|       No|
| 32|       Canada|  Male|       No|female|       No|
| 32|       Canada|  Male|       No|Female|      Yes|
| 32|       Canada|  Male|       No|Female|      Yes|
| 32|       Canada|  Male|  

In [18]:
# Question2 part2
spark.sql("select * from Survey ORDER BY Timestamp limit 13").createTempView("Test")

In [19]:
spark.sql("select * from Test order by Timestamp desc limit 1").show()

+-------------------+---+------+-------------+-----+-------------+--------------+---------+--------------+------------+-----------+------------+--------+------------+----------------+---------+----------+------------------+-------------------------+-----------------------+---------+----------+-----------------------+---------------------+------------------+---------------+--------+
|          Timestamp|Age|Gender|      Country|state|self_employed|family_history|treatment|work_interfere|no_employees|remote_work|tech_company|benefits|care_options|wellness_program|seek_help| anonymity|             leave|mental_health_consequence|phys_health_consequence|coworkers|supervisor|mental_health_interview|phys_health_interview|mental_vs_physical|obs_consequence|comments|
+-------------------+---+------+-------------+-----+-------------+--------------+---------+--------------+------------+-----------+------------+--------+------------+----------------+---------+----------+------------------+-------

In [20]:
##Bonus Question1

In [21]:
from pyspark.sql.types import *
schema = StructType([
 StructField("Timestamp", StringType(), True),
 StructField("Age", IntegerType(), True),
 StructField("Gender", StringType(), True),
 StructField("Country", StringType(), True),
 StructField("state", StringType(), True),
 StructField("self_employed", StringType(), True),
 StructField("family_history", StringType(), True),
 StructField("treatment", StringType(), True),
 StructField("work_interfere", StringType(), True),
 StructField("no_employees", StringType(), True),
 StructField("remote_work", StringType(), True),
 StructField("tech_company", StringType(), True),
 StructField("benefits", StringType(), True),
 StructField("care_options", StringType(), True),
 StructField("wellness_program", StringType(), True),
 StructField("seek_help", StringType(), True),
 StructField("anonymity", StringType(), True),
 StructField("leave", StringType(), True),
 StructField("mental_health_consequence", StringType(), True),
 StructField("phys_health_consequence", StringType(), True),
 StructField("coworkers", StringType(), True),
 StructField("supervisor", StringType(), True),
 StructField("mental_health_interview", StringType(), True),
 StructField("phys_health_interview", StringType(), True),
 StructField("mental_vs_physical", StringType(), True),
 StructField("obs_consequence", StringType(), True),
 StructField("comments", StringType(), True)

])

In [22]:
dfnew=spark.read.schema(schema).option("header", "true").option("mode", "DROPMALFORMED").csv("survey.csv")

In [23]:
dfnew.collect()

[Row(Timestamp='2014-08-27 11:29:31', Age=37, Gender='Female', Country='United States', state='IL', self_employed='NA', family_history='No', treatment='Yes', work_interfere='Often', no_employees='6-25', remote_work='No', tech_company='Yes', benefits='Yes', care_options='Not sure', wellness_program='No', seek_help='Yes', anonymity='Yes', leave='Somewhat easy', mental_health_consequence='No', phys_health_consequence='No', coworkers='Some of them', supervisor='Yes', mental_health_interview='No', phys_health_interview='Maybe', mental_vs_physical='Yes', obs_consequence='No', comments='NA'),
 Row(Timestamp='2014-08-27 11:29:37', Age=44, Gender='M', Country='United States', state='IN', self_employed='NA', family_history='No', treatment='No', work_interfere='Rarely', no_employees='More than 1000', remote_work='No', tech_company='No', benefits="Don't know", care_options='No', wellness_program="Don't know", seek_help="Don't know", anonymity="Don't know", leave="Don't know", mental_health_consequ

In [24]:
dfnew.columns

['Timestamp',
 'Age',
 'Gender',
 'Country',
 'state',
 'self_employed',
 'family_history',
 'treatment',
 'work_interfere',
 'no_employees',
 'remote_work',
 'tech_company',
 'benefits',
 'care_options',
 'wellness_program',
 'seek_help',
 'anonymity',
 'leave',
 'mental_health_consequence',
 'phys_health_consequence',
 'coworkers',
 'supervisor',
 'mental_health_interview',
 'phys_health_interview',
 'mental_vs_physical',
 'obs_consequence',
 'comments']

In [25]:
## Bonus question3

In [26]:

## correlation on same variable
pearsoncorr = dfnew.corr('Age','Age',method='pearson')
pearsoncorr

1.0

In [27]:
#String indexing
from pyspark.ml.feature import StringIndexer

In [28]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(df) for column in list(set(dfnew.columns)) ]


pipeline = Pipeline(stages=indexers)
df_r = pipeline.fit(dfnew).transform(dfnew)

df_r.show()

+-------------------+---+------+--------------+-----+-------------+--------------+---------+--------------+--------------+-----------+------------+----------+------------+----------------+----------+----------+------------------+-------------------------+-----------------------+------------+------------+-----------------------+---------------------+------------------+---------------+--------------------+------------------+------------+--------------+-----------------------------+---------------+---------------------------+------------------------+---------------+--------------------+-------------------+---------------+---------------+--------------+------------------+---------------+---------+--------------------+-----------+-------------------------------+---------------------+-----------+------------------+-----------------------------+-------------+----------------+----------------------+-----------------+
|          Timestamp|Age|Gender|       Country|state|self_employed|family_his

In [29]:
df_r.createOrReplaceTempView("NewSurvey")

In [30]:
spark.sql("select * from NewSurvey").show()

+-------------------+---+------+--------------+-----+-------------+--------------+---------+--------------+--------------+-----------+------------+----------+------------+----------------+----------+----------+------------------+-------------------------+-----------------------+------------+------------+-----------------------+---------------------+------------------+---------------+--------------------+------------------+------------+--------------+-----------------------------+---------------+---------------------------+------------------------+---------------+--------------------+-------------------+---------------+---------------+--------------+------------------+---------------+---------+--------------------+-----------+-------------------------------+---------------------+-----------+------------------+-----------------------------+-------------+----------------+----------------------+-----------------+
|          Timestamp|Age|Gender|       Country|state|self_employed|family_his

In [31]:
type(dfnew)

pyspark.sql.dataframe.DataFrame

In [32]:
df_r.columns


['Timestamp',
 'Age',
 'Gender',
 'Country',
 'state',
 'self_employed',
 'family_history',
 'treatment',
 'work_interfere',
 'no_employees',
 'remote_work',
 'tech_company',
 'benefits',
 'care_options',
 'wellness_program',
 'seek_help',
 'anonymity',
 'leave',
 'mental_health_consequence',
 'phys_health_consequence',
 'coworkers',
 'supervisor',
 'mental_health_interview',
 'phys_health_interview',
 'mental_vs_physical',
 'obs_consequence',
 'comments',
 'tech_company_index',
 'Gender_index',
 'comments_index',
 'mental_health_interview_index',
 'seek_help_index',
 'phys_health_interview_index',
 'mental_vs_physical_index',
 'coworkers_index',
 'work_interfere_index',
 'self_employed_index',
 'anonymity_index',
 'Timestamp_index',
 'benefits_index',
 'care_options_index',
 'treatment_index',
 'Age_index',
 'family_history_index',
 'leave_index',
 'mental_health_consequence_index',
 'obs_consequence_index',
 'state_index',
 'no_employees_index',
 'phys_health_consequence_index',
 'Co

In [33]:
## Bonus question2  Covariance
pearsoncovnew = df_r.cov('Age_index','treatment_index')
pearsoncovnew

-0.2522750182444136

In [34]:
## Bonus question3  Correlation
pearsoncorrnewtreat = df_r.corr('Age_index','treatment_index',method='pearson')
pearsoncorrnewtreat

-0.05824190070043755

In [35]:
## Bonus question3  Correlation
pearsoncorrnew = df_r.corr('Age_index','remote_work_index',method='pearson')
pearsoncorrnew

0.11758477982766366

In [36]:
#question 1  part2
df_r.write.option("header", "true").csv("spark_survey124.csv")